In [1]:
#!pip3 install twilio

In [3]:
import os
from twilio.rest import Client
from twilio_config import *
import time

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json


import pandas as pd
import requests
from bs4  import BeautifulSoup
from tqdm import tqdm

from datetime import datetime


# Armado de la URL

In [4]:
query = 'Panama'
api_key = API_KEY_WAPI

url_clima = 'http://api.weatherapi.com/v1/forecast.json?key='+api_key+'&q='+query+'&days=1&aqi=no&alerts=no'
url_clima

'http://api.weatherapi.com/v1/forecast.json?key=a665e0ba41ba413bbb7213605252402&q=Panama&days=1&aqi=no&alerts=no'

In [5]:
response = requests.get(url_clima).json()

In [6]:
response

{'location': {'name': 'Panama City',
  'region': 'Panama',
  'country': 'Panama',
  'lat': 8.967,
  'lon': -79.533,
  'tz_id': 'America/Panama',
  'localtime_epoch': 1740433669,
  'localtime': '2025-02-24 16:47'},
 'current': {'last_updated_epoch': 1740433500,
  'last_updated': '2025-02-24 16:45',
  'temp_c': 31.1,
  'temp_f': 88.0,
  'is_day': 1,
  'condition': {'text': 'Partly cloudy',
   'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png',
   'code': 1003},
  'wind_mph': 11.9,
  'wind_kph': 19.1,
  'wind_degree': 319,
  'wind_dir': 'NW',
  'pressure_mb': 1009.0,
  'pressure_in': 29.8,
  'precip_mm': 0.0,
  'precip_in': 0.0,
  'humidity': 59,
  'cloud': 50,
  'feelslike_c': 35.0,
  'feelslike_f': 95.0,
  'windchill_c': 29.4,
  'windchill_f': 85.0,
  'heatindex_c': 31.9,
  'heatindex_f': 89.3,
  'dewpoint_c': 20.5,
  'dewpoint_f': 68.9,
  'vis_km': 10.0,
  'vis_miles': 6.0,
  'uv': 2.3,
  'gust_mph': 16.4,
  'gust_kph': 26.4},
 'forecast': {'forecastday': [{'date': '2025-02-24',


In [7]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [8]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [10]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [ ]:
response['forecast']['forecastday'][0]['hour'][0]

In [12]:
response['forecast']['forecastday'][0]['hour'][0]['time'].split()[0]

'2025-02-24'

In [14]:
int(response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0])

1

In [15]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

0

In [16]:
response['forecast']['forecastday'][0]['hour'][2]['chance_of_rain']

0

# Dataframe

In [20]:
def get_forecast(response,i):

    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    tempe = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_rain = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']

    return fecha,hora,condicion,tempe,rain,prob_rain

In [21]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])),colour = 'green'):

    datos.append(get_forecast(response,i))


100%|██████████| 24/24 [00:00<00:00, 113232.05it/s]


In [25]:
datos[0]

('2025-02-24', 0, 'Patchy rain nearby', 24.3, 0, 67)

In [26]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','prob_lluvia']
df = pd.DataFrame(datos,columns=col)
df = df.sort_values(by = 'Hora',ascending = True)
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
0,2025-02-24,0,Patchy rain nearby,24.3,0,67
1,2025-02-24,1,Patchy rain nearby,23.7,0,70
2,2025-02-24,2,Clear,23.4,0,0
3,2025-02-24,3,Clear,23.2,0,0
4,2025-02-24,4,Clear,23.1,0,0
5,2025-02-24,5,Patchy rain nearby,23.0,1,85
6,2025-02-24,6,Clear,22.9,0,0
7,2025-02-24,7,Patchy rain nearby,23.2,1,72
8,2025-02-24,8,Patchy rain nearby,25.0,1,86
9,2025-02-24,9,Patchy rain nearby,26.7,1,77


In [ ]:
# data_cli = {
#     'Fecha':datos[0][0],
#     'Hora':datos[1][],
#     'Tiempo':condicion,
#     'Temperatura':tempe,
#     'LLoverá':rain,
#     'prob_lluvia':prob_rain

# }

# df = pd.DataFrame(data_cli)
# df = df.sort_values(by = 'Hora',ascending = True)

In [28]:
df[df['Lluvia']==1]

,Fecha,Hora,Condicion,Temperatura,Lluvia,prob_lluvia
5,2025-02-24,5,Patchy rain nearby,23.0,1,85
7,2025-02-24,7,Patchy rain nearby,23.2,1,72
8,2025-02-24,8,Patchy rain nearby,25.0,1,86
9,2025-02-24,9,Patchy rain nearby,26.7,1,77
10,2025-02-24,10,Patchy rain nearby,28.2,1,81
11,2025-02-24,11,Patchy rain nearby,29.4,1,100
13,2025-02-24,13,Patchy rain nearby,29.8,1,89
15,2025-02-24,15,Patchy rain nearby,29.0,1,73


In [29]:
df_rain =  df[(df['Lluvia']==1) & (df['Hora']>6) & (df['Hora']< 22)]
df_rain = df_rain[['Hora','Condicion']]
df_rain.set_index('Hora', inplace = True)

In [30]:
df['Fecha'][0]

'2025-02-24'

In [29]:
df_rain

,Condicion
Hora,
14,Patchy rain possible
16,Patchy rain possible


In [30]:
'\nHola! \n\n\n El pronostico del tiempo hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain)

'\nHola! \n\n\n El pronostico del tiempo hoy 2022-09-15 en Bogotá es : \n\n\n                  Condicion\nHora                      \n14    Patchy rain possible\n16    Patchy rain possible'

In [27]:
PHONE_NUMBER

'+12622052596'

# Mensaje SMS desde Twilio

In [ ]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                     body='\nHola! \n\n\n El pronostico de lluvia hoy '+ df['Fecha'][0] +' en ' + query +' es : \n\n\n ' + str(df_rain),
                     from_=PHONE_NUMBER,
                     to='+573222007879'
                 )

print('Mensaje Enviado ' + message.sid)

Mensaje Enviado SMbc309ab2afc09ea1f87a7eb64fb8fdda


# Challenge 

* Extrae el valor del dolar en tu país y el top 10 de criptomonedas con su respectiva valoración
* Ahora envia un mensaje diarío a tu Whatsapp usando Twilio

**hint 💡** Investiga que API's gratuitas existen para consultar estos datos



<img src="WhatsApp Image 2022-09-13 at 9.12.18 AM.jpeg" width="200" height="200" />